#### Run install cells only if your env is missing it

In [1]:
!pip install folium --upgrade

In [2]:
!pip install geocoder

In [1]:
from numpy import radians, cos, sin, sqrt
from numpy import arcsin as asin
import pandas as pd
import numpy as np
import re
import os
import glob
from datetime import datetime

import requests
import folium
from folium import plugins
import ipywidgets
import geocoder
from folium.features import DivIcon

In [2]:
template_path_str = input('Enter path to paylah.csv')
local_path1 = "\\".join(template_path_str.split("\\")[:2]) 
local_path2 = "\\".join(template_path_str.split("\\")[3:])
local_path = template_path_str
fname = 'paylah.csv'
local_path = local_path + "\\" + fname
local_path

Enter path to paylah.csvC:\Users\shahril.mustafa\Documents\byPurpOrFunc\Pers


'C:\\Users\\shahril.mustafa\\Documents\\byPurpOrFunc\\Pers\\paylah.csv'

In [3]:
df_paylah = pd.read_csv(local_path)
df_paylah.head()

,Column1,Column2.1,Column2.2
0,101 YISHUN CS,101 YISHUN AVE 5,760101
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115


In [4]:
df_paylah.rename(columns={"Column2.2": "Postal Code"}, inplace=True)
df_paylah['Postal Code'] = df_paylah['Postal Code'].map(str)
df_paylah['Postal Code'] = df_paylah['Postal Code'].apply(lambda x: x.rjust(6, '0'))
print(len(df_paylah))
df_paylah.head()

884


,Column1,Column2.1,Postal Code
0,101 YISHUN CS,101 YISHUN AVE 5,760101
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115


In [6]:
# type(response_me.json()['results'])
found = 0
while found == 0:
    my_loc_pc = input('Enter your SG postal code eg. 188720 >>> ')
    url_me = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={my_loc_pc}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response_me = requests.request("GET", url_me)
    print(response_me.text)    
    found = len(response_me.json()['results'])

Enter your SG postal code eg. 188720 >>> 188720
{
  "found": 2,
  "totalNumPages": 1,
  "pageNum": 1,
  "results": [
    {
      "SEARCHVAL": "ODEON TOWERS",
      "BLK_NO": "331",
      "ROAD_NAME": "NORTH BRIDGE ROAD",
      "BUILDING": "ODEON TOWERS",
      "ADDRESS": "331 NORTH BRIDGE ROAD ODEON TOWERS SINGAPORE 188720",
      "POSTAL": "188720",
      "X": "30219.3706113212",
      "Y": "30950.5209991969",
      "LATITUDE": "1.29617995172042",
      "LONGITUDE": "103.853260592876"
    },
    {
      "SEARCHVAL": "HIGH COMMISSION OF THE REPUBLIC OF SOUTH AFRICA",
      "BLK_NO": "331",
      "ROAD_NAME": "NORTH BRIDGE ROAD",
      "BUILDING": "HIGH COMMISSION OF THE REPUBLIC OF SOUTH AFRICA",
      "ADDRESS": "331 NORTH BRIDGE ROAD HIGH COMMISSION OF THE REPUBLIC OF SOUTH AFRICA SINGAPORE 188720",
      "POSTAL": "188720",
      "X": "30217.6978512006",
      "Y": "30935.2762870546",
      "LATITUDE": "1.29604208403261",
      "LONGITUDE": "103.85324556132"
    }
  ]
}


In [7]:
lat_me = response_me.json()['results'][0]["LATITUDE"]
lng_me = response_me.json()['results'][0]["LONGITUDE"]

In [8]:
map_zoo = folium.Map()
map_zoo = folium.Map(location=[float(lat_me), float(lng_me)], zoom_start=17)
map_zoo

In [9]:
def get_LATLONG(astr):
    url_me = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={astr}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response_me = requests.request("GET", url_me)
    if response_me.json()['found'] < 1:
        print(astr)
    return response_me.json()['results'][0]["LATITUDE"], response_me.json()['results'][0]["LONGITUDE"]

In [10]:
# Known erroneous data in PDF/CSV
df_paylah['Postal Code'].replace('760054', '760654', inplace=True)
df_paylah['Postal Code'].replace('370218', '372018', inplace=True)
df_paylah['Postal Code'].replace('310102', '310212', inplace=True)

In [11]:
df_paylah['LATLONG'] = df_paylah['Postal Code'].apply(get_LATLONG)
df_paylah.head()

,Column1,Column2.1,Postal Code,LATLONG
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"(1.43053231016112, 103.827618665577)"
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"(1.35370313485788, 103.890680820842)"
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"(1.28000480135122, 103.82601329621)"
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"(1.28550445567433, 103.821823256087)"
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"(1.44608188224858, 103.828739863564)"


In [12]:
df_paylah["LATLONG"] = df_paylah["LATLONG"].map(str)
df_paylah[['LAT', 'LONG']] = df_paylah["LATLONG"].str.split(pat = ', ', expand = True, n=1)
df_paylah['LAT'] = df_paylah['LAT'].str.replace(r'^..', '',regex=True).str.replace(r'\'$', '',regex=True)
df_paylah['LONG'] = df_paylah['LONG'].str.replace(r'^.', '',regex=True).str.replace(r'\'.$', '',regex=True)
df_paylah.head()

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"('1.43053231016112', '103.827618665577')",1.43053231016112,103.827618665577
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"('1.35370313485788', '103.890680820842')",1.35370313485788,103.890680820842
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"('1.28000480135122', '103.82601329621')",1.28000480135122,103.82601329621
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"('1.28550445567433', '103.821823256087')",1.28550445567433,103.821823256087
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"('1.44608188224858', '103.828739863564')",1.44608188224858,103.828739863564


In [13]:
def haversine_astr(astr):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon2=float(lng_me)
    lat2=float(lat_me)
    # convert decimal degrees to radians 
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    lat1, lon1, lat2, lon2 = map(radians, [astr['LONG'], astr['LAT'], lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [14]:
df_paylah['LAT'] = df_paylah['LAT'].map(float)
df_paylah['LONG'] = df_paylah['LONG'].map(float)
df_paylah['haversine'] = df_paylah.apply(haversine_astr, axis=1)
df_paylah.head()

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG,haversine
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"('1.43053231016112', '103.827618665577')",1.430532,103.827619,4.571805
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"('1.35370313485788', '103.890680820842')",1.353703,103.890681,4.434538
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"('1.28000480135122', '103.82601329621')",1.280005,103.826013,3.060156
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"('1.28550445567433', '103.821823256087')",1.285504,103.821823,3.507183
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"('1.44608188224858', '103.828739863564')",1.446082,103.828740,4.830598


In [15]:
df_paylah.haversine.min()

0.04140941322799226

In [16]:
df_paylah.haversine.max()

25.212588974905252

In [17]:
df_paylah.haversine.mean()

6.709724275527577

In [18]:
dist_ = 0.35
df_local = df_paylah.loc[df_paylah.haversine<dist_].copy()
len(df_local)

5

In [19]:
df_local

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG,haversine
45,3838 EATING PLACE,269 QUEEN ST,180269,"('1.30048013687566', '103.85391368667')",1.300480,103.853914,0.135580
124,ALBERT CENTRE FOOD CENTRE,270 QUEEN ST,180270,"('1.30110201573423', '103.854115779087')",1.301102,103.854116,0.161915
176,BGAIN 261 EATING HOUSE,261 WATERLOO ST,180261,"('1.29869445251522', '103.852082013485')",1.298694,103.852082,0.147160
230,CHANG CHENG (269),269 QUEEN ST,180269,"('1.30048013687566', '103.85391368667')",1.300480,103.853914,0.135580
273,COFFEE EXPRESS 2000 (231),231 BAIN ST,180231,"('1.29680870763696', '103.853601209503')",1.296809,103.853601,0.041409


In [20]:
folium.Marker(location=[float(lat_me), float(lng_me)], popup='... are here', tooltip='You').add_to(map_zoo)
# for (index, row) in df_paylah.iterrows():
for (index, row) in df_local.iterrows():
  folium.Marker(location=[row.loc['LAT'], row.loc['LONG']], popup=row.loc['Column1'], 
      icon=plugins.BeautifyIcon(#number=row.loc['Column1'],#i.icon_num,
                        icon='cutlery',
                        border_color='blue',
                        border_width=1,
                        text_color='red'#,
#                         inner_icon_style='margin-top:3px;')#).add_to(map_enum_icons)
                        )#).add_to(map_enum_icons)

        ).add_to(map_zoo)
  map_zoo.add_child(folium.CircleMarker(location=[row.loc['LAT'], row.loc['LONG']], radius=15))

# display map
map_zoo

In [73]:
map_zoo.save('myloc.html')